In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('SFM20-21data.txt')
winter2 = pd.read_csv ('SFM21-22data.txt')
winter3 = pd.read_csv ('SFM22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
precip_days3[676:690]

,station,datetime,precip(mm),wxcodes
1859,SFM,2023-01-25 21:17:00,0.0001,-SN BR
1860,SFM,2023-01-25 21:39:00,0.0001,-SN BR
1861,SFM,2023-01-25 21:56:00,0.0001,-SN BR
1862,SFM,2023-01-25 22:49:00,0.0001,-SN BR
1863,SFM,2023-01-25 22:56:00,0.0001,-SN BR
1864,SFM,2023-01-25 23:18:00,0.0001,-SN BR
1865,SFM,2023-01-25 23:46:00,0.0001,-SN BR
1866,SFM,2023-01-25 23:56:00,0.0001,-SN BR
1867,SFM,2023-01-26 00:12:00,0.0001,-SN BR
1868,SFM,2023-01-26 00:31:00,0.0001,-SN BR


In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
sfm = LocationInfo(
    'Sanford', 'Maine', 
    latitude= 43.4388,
    longitude= -70.7751,
    timezone='America/Connecticut'
)

sfm

LocationInfo(name='Sanford', region='Maine', timezone='America/Connecticut', latitude=43.4388, longitude=-70.7751)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(sfm.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(sfm.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(sfm.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:56:49  2020-12-01 16:07:40  2020-12-01
1   2020-12-02 06:57:53  2020-12-02 16:07:23  2020-12-02
2   2020-12-03 06:58:56  2020-12-03 16:07:09  2020-12-03
3   2020-12-04 06:59:58  2020-12-04 16:06:56  2020-12-04
4   2020-12-05 07:00:59  2020-12-05 16:06:47  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:27:25  2021-02-24 17:25:41  2021-02-24
86  2021-02-25 06:25:49  2021-02-25 17:26:59  2021-02-25
87  2021-02-26 06:24:11  2021-02-26 17:28:16  2021-02-26
88  2021-02-27 06:22:33  2021-02-27 17:29:33  2021-02-27
89  2021-02-28 06:20:54  2021-02-28 17:30:50  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:56:33  2021-12-01 16:07:45  2021-12-01
1   2021-12-02 06:57:38  2021-12-02 16:07:27  2021-12-02
2   2021-12-03 06:58:41  2021-12-03 16:07:12  2021-12-03
3   2021-12-04 06:59:44  2021-12-04 16:06:59  2021-12-04
4   2021-12-05 07:00:44  2021-12-05 16:06:48  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:27:49  2022-02-24 17:25:22  2022-02-24
86  2022-02-25 06:26:12  2022-02-25 17:26:40  2022-02-25
87  2022-02-26 06:24:35  2022-02-26 17:27:57  2022-02-26
88  2022-02-27 06:22:57  2022-02-27 17:29:14  2022-02-27
89  2022-02-28 06:21:18  2022-02-28 17:30:31  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:56:17  2022-12-01 16:07:49  2022-12-01
1   2022-12-02 06:57:22  2022-12-02 16:07:31  2022-12-02
2   2022-12-03 06:58:26  2022-12-03 16:07:15  2022-12-03
3   2022-12-04 06:59:29  2022-12-04 16:07:01  2022-12-04
4   2022-12-05 07:00:30  2022-12-05 16:06:50  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:28:12  2023-02-24 17:25:03  2023-02-24
86  2023-02-25 06:26:36  2023-02-25 17:26:21  2023-02-25
87  2023-02-26 06:24:59  2023-02-26 17:27:38  2023-02-26
88  2023-02-27 06:23:21  2023-02-27 17:28:55  2023-02-27
89  2023-02-28 06:21:42  2023-02-28 17:30:12  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFM,2020-12-01 00:56:00,0.0000,RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
1,SFM,2020-12-01 01:56:00,0.0000,RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
2,SFM,2020-12-01 02:56:00,0.0000,-RA,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
3,SFM,2020-12-01 03:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
4,SFM,2020-12-01 04:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
...,...,...,...,...,...,...,...
333,SFM,2021-02-27 18:56:00,3.8100,-RA BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
334,SFM,2021-02-27 19:56:00,2.5400,-RA BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
335,SFM,2021-02-27 20:56:00,0.7600,-RA BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
336,SFM,2021-02-27 21:56:00,0.5100,-RA FG,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFM,2021-12-02 09:56:00,0.0001,UP,2021-12-02,2021-12-02 06:57:38,2021-12-02 16:07:27
1,SFM,2021-12-02 10:56:00,0.0001,UP,2021-12-02,2021-12-02 06:57:38,2021-12-02 16:07:27
2,SFM,2021-12-02 11:56:00,1.5200,UP BR,2021-12-02,2021-12-02 06:57:38,2021-12-02 16:07:27
3,SFM,2021-12-02 12:56:00,1.2700,UP BR,2021-12-02,2021-12-02 06:57:38,2021-12-02 16:07:27
4,SFM,2021-12-02 13:56:00,1.5200,-RA BR,2021-12-02,2021-12-02 06:57:38,2021-12-02 16:07:27
...,...,...,...,...,...,...,...
382,SFM,2022-02-25 23:56:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:26:12,2022-02-25 17:26:40
383,SFM,2022-02-26 00:56:00,0.0001,-SN,2022-02-26,2022-02-26 06:24:35,2022-02-26 17:27:57
384,SFM,2022-02-26 01:56:00,0.2500,-SN,2022-02-26,2022-02-26 06:24:35,2022-02-26 17:27:57
385,SFM,2022-02-26 03:56:00,0.5100,-SN,2022-02-26,2022-02-26 06:24:35,2022-02-26 17:27:57


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFM,2022-12-01 00:56:00,6.3500,RA BR,2022-12-01,2022-12-01 06:56:17,2022-12-01 16:07:49
1,SFM,2022-12-01 01:56:00,0.5100,-RA,2022-12-01,2022-12-01 06:56:17,2022-12-01 16:07:49
2,SFM,2022-12-01 04:56:00,0.0001,-RA,2022-12-01,2022-12-01 06:56:17,2022-12-01 16:07:49
3,SFM,2022-12-01 18:56:00,0.0001,-RA,2022-12-01,2022-12-01 06:56:17,2022-12-01 16:07:49
4,SFM,2022-12-03 08:56:00,0.0001,-RA,2022-12-03,2022-12-03 06:58:26,2022-12-03 16:07:15
...,...,...,...,...,...,...,...
396,SFM,2023-02-26 18:56:00,0.2500,-SN BR,2023-02-26,2023-02-26 06:24:59,2023-02-26 17:27:38
397,SFM,2023-02-26 19:56:00,0.0001,-SN BR,2023-02-26,2023-02-26 06:24:59,2023-02-26 17:27:38
398,SFM,2023-02-26 20:56:00,0.0001,-SN,2023-02-26,2023-02-26 06:24:59,2023-02-26 17:27:38
399,SFM,2023-02-26 21:56:00,0.0001,-SN,2023-02-26,2023-02-26 06:24:59,2023-02-26 17:27:38


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,SFM,2020-12-01 07:56:00,0.0000,RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
8,SFM,2020-12-01 09:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
9,SFM,2020-12-01 10:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
11,SFM,2020-12-05 11:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:00:59,2020-12-05 16:06:47
12,SFM,2020-12-05 12:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:00:59,2020-12-05 16:06:47
...,...,...,...,...,...,...,...
327,SFM,2021-02-27 12:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
328,SFM,2021-02-27 13:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
329,SFM,2021-02-27 14:56:00,1.5200,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
330,SFM,2021-02-27 15:56:00,1.7800,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,SFM,2020-12-01 09:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40,2020-12-01 07:56:49,2020-12-01 15:07:40
9,SFM,2020-12-01 10:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40,2020-12-01 07:56:49,2020-12-01 15:07:40
11,SFM,2020-12-05 11:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:00:59,2020-12-05 16:06:47,2020-12-05 08:00:59,2020-12-05 15:06:47
12,SFM,2020-12-05 12:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:00:59,2020-12-05 16:06:47,2020-12-05 08:00:59,2020-12-05 15:06:47
13,SFM,2020-12-05 13:56:00,2.2900,-RA,2020-12-05,2020-12-05 07:00:59,2020-12-05 16:06:47,2020-12-05 08:00:59,2020-12-05 15:06:47
...,...,...,...,...,...,...,...,...,...
318,SFM,2021-02-20 13:56:00,0.0001,-SN,2021-02-20,2021-02-20 06:33:42,2021-02-20 17:20:28,2021-02-20 07:33:42,2021-02-20 16:20:28
327,SFM,2021-02-27 12:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33,2021-02-27 07:22:33,2021-02-27 16:29:33
328,SFM,2021-02-27 13:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33,2021-02-27 07:22:33,2021-02-27 16:29:33
329,SFM,2021-02-27 14:56:00,1.5200,-SN BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33,2021-02-27 07:22:33,2021-02-27 16:29:33


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4621726315789475
2021-2022: 0.38581780821917805
2022-2023: 0.8358991150442476


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,SFM,2020-12-01 00:56:00,0.0000,RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
1,SFM,2020-12-01 01:56:00,0.0000,RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
2,SFM,2020-12-01 02:56:00,0.0000,-RA,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
3,SFM,2020-12-01 03:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
4,SFM,2020-12-01 04:56:00,0.0000,-RA BR,2020-12-01,2020-12-01 06:56:49,2020-12-01 16:07:40
...,...,...,...,...,...,...,...
333,SFM,2021-02-27 18:56:00,3.8100,-RA BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
334,SFM,2021-02-27 19:56:00,2.5400,-RA BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
335,SFM,2021-02-27 20:56:00,0.7600,-RA BR,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33
336,SFM,2021-02-27 21:56:00,0.5100,-RA FG,2021-02-27,2021-02-27 06:22:33,2021-02-27 17:29:33


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_6432\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.47989327731092435
2021-2022: 0.1858158415841584
2022-2023: 0.35837663551401866


In [68]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('sfm.csv', index=False)